In [1]:
import math
from typing import List, Union

In [2]:
def parse_bus_ids(raw_bus_ids: str) -> List[Union[int, str]]:
    bus_id_strs = raw_bus_ids.strip().split(",")
    bus_ids = []
    # lol
    for bus_id in bus_id_strs:
        if bus_id == "x":
            bus_ids.append(bus_id)
        else:
            bus_ids.append(int(bus_id))
    return bus_ids

In [3]:
filename = "day-13-input.txt"

with open(filename) as file:
    lines = file.readlines()

assert len(lines) == 2
estimated_ts = int(lines[0].strip())
bus_ids = parse_bus_ids(lines[1])

# Part 1

In [4]:
earliest_ts = float("inf")
earliest_bus_id = -1

for bus_id in bus_ids:
    if bus_id == "x":
        continue
    ts_multiple = int(bus_id)
    ts = math.ceil(estimated_ts / ts_multiple) * ts_multiple
    
    if ts < earliest_ts:
        earliest_bus_id = ts_multiple
        earliest_ts = ts

print("Bus ID:", earliest_bus_id)
print("Departure timestamp:", earliest_ts)
ts_diff = earliest_ts - estimated_ts
print(f"Answer is: {earliest_bus_id} * {ts_diff} = {earliest_bus_id*ts_diff}")

Bus ID: 29
Departure timestamp: 1001805
Answer is: 29 * 7 = 203


# Part 2: Number Theory Method

Wow, this required me to do some serious brushing up on Number Theory, which I am *really* rusty on.

The solution here relies on all pairings of bus ids being co-prime (the next cell checks for this).

In [5]:
# First, check that all the numbers are pairwise co-prime.

import itertools


def check_gcds(bus_ids):
    bus_ids = [bus_id for bus_id in bus_ids if isinstance(bus_id, int)]
    for combo in itertools.combinations(bus_ids, 2):
        if math.gcd(*combo) != 1: 
            return False
        
    return True

assert check_gcds(bus_ids)

Once that's verified, we can use the [Chinese Remainder Theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem) to figure out the appropriate timestamp. To get the final answer of timestamp $T$, we'd basically be solving this system of linear congruences:

$$\begin{align}
  T - \text{bus_index}_0 &\equiv 0 \pmod {\text{bus_id}_0}\\
  T - \text{bus_index}_1 &\equiv 0 \pmod {\text{bus_id}_1}\\
  T - \text{bus_index}_2 &\equiv 0 \pmod {\text{bus_id}_2}\\
  &\,\,\,\vdots
\end{align}$$

This can be re-rwitten as:
$$\begin{align}
  T &\equiv -\text{bus_index}_0 \pmod {\text{bus_id}_0}\\
  T &\equiv -\text{bus_index}_1 \pmod {\text{bus_id}_1}\\
  T &\equiv -\text{bus_index}_2 \pmod {\text{bus_id}_2}\\
  &\,\,\,\vdots
\end{align}$$

I can't say I've had the patience to read the Wikipedia article for the Chinese Remainder Theorem, but luckily I had notes about from my AoPS days... which I still spent a good chunk of my Sunday reading.

In any case, here's the algorithm I used for the Chinese Remainder Theorem:

1. For every congruence, find the product $P$ of all the other congruences' moduli (all the other $\text{bus_id}$s). Then multiply $P$ by the multiplicative inverse of $P$ for this congruence's modulus and by the right side of the equation (I forget the mathematical term for it; it's equivalent to $-\text{bus_index}_i$ in this problem).
2. Add up all your results from step 1.
3. Reduce step 2's result by modulo $Q$, where $Q$ is the product of all the moduli (in this case, all the $\text{bus_id}$s).

You can convince yourself this works by taking the *unreduced* addition expressions at step 2 (don't add the results from step 1 together and also keep the bus indices factored out in each product from step 1) and seeing what happens to every term when you reduce this expression by modulo $\text{bus_id}$ for every $\text{bus_id}$.

(And if what I'm writing doesn't make sense... I'm not spending a whole lot of time on this explanation, so you'll need to find some other resource that does make sense for you. 🙂)

In [6]:
def multiply_bus_ids(bus_ids: List[Union[int, str]], exclude_idx: int) -> int:
    """Multiplies non-x bus ids, with option to exclude a bus id."""
    product = 1
    for idx, bus_id in enumerate(bus_ids):
        if bus_id == "x" or idx == exclude_idx:
            continue
        product *= bus_id
    return product

In [7]:
def multiplicative_inverse(num: int, mod: int) -> int:
    """
    The multiplicative inverse (b) of a mod m fulfills this expression:
    ab ≡ 1 (mod m)
    (If the symbol doesn't show up, it's the "three lines" sign similar to an equals sign.)
    """
    # brute force, wheeeee!
    for i in range(mod):
        if num * i % mod == 1:
            return i

In [8]:
def find_timestamp(bus_ids: List[Union[int, str]]) -> int:
    
    answer = 0
    
    for idx, bus_id in enumerate(bus_ids):
        if bus_id == "x":
            continue
        
        other_ids_product = multiply_bus_ids(bus_ids, idx)
        answer += -idx * other_ids_product * multiplicative_inverse(other_ids_product, bus_id)
    
    all_product = multiply_bus_ids(bus_ids, None)
    
    return answer % all_product

### Test Cases:

In [9]:
test_cases = [
    ("7,13,x,x,59,x,31,19", 1068781),
    ("17,x,13,19", 3417),
    ("67,7,59,61", 754018),
    ("67,x,7,59,61", 779210),
    ("67,7,x,59,61", 1261476),
    ("1789,37,47,1889", 1202161486),
]

for bus_ids_str, answer in test_cases:
    ts = find_timestamp(parse_bus_ids(bus_ids_str))
    assert ts == answer
    sign = "==" if ts == answer else "!="
    print(f"{ts} {sign} {answer}")

1068781 == 1068781
3417 == 3417
754018 == 754018
779210 == 779210
1261476 == 1261476
1202161486 == 1202161486


### Actual Answer:

In [10]:
print("The earliest timestamp for Part 2 is:", find_timestamp(bus_ids))

The earliest timestamp for Part 2 is: 905694340256752
